In [1]:
%matplotlib inline

In [2]:
from bb_binary import FrameContainer, Repository, load_frame_container
from datetime import datetime, timedelta
from collections import namedtuple
from collections import Counter
import multiprocessing
import sqlite3
import matplotlib

import numpy as np
import pandas as pd
import json
import tables

from collections import Counter

from pandas import DataFrame, Series

In [3]:
def get_detected_id(bits):
    binary_id = (bits>0.5)*1
    decimal_id = int(''.join([str(c) for c in binary_id.tolist()[:11]]), 2)
    if ((sum(binary_id) % 2) == 1):
        decimal_id += 2048

    return decimal_id

In [4]:
def get_confidence(bits):
    return np.min(np.abs(0.5 - bits)) * 2

In [5]:
# Dezimale ID ausrechnen und an DataFrame angaengen
def calcIds(df, threshold):
    df.decodedId = df.decodedId.apply(lambda x: np.array(x)/255)
    df = df.assign(confidence = df.decodedId.apply(get_confidence))
    df = df[df.confidence >= threshold]
    df = df.assign(id = df.decodedId.apply(get_detected_id))

    return df

In [6]:
Detection = namedtuple(
    'Detection',
    ['idx', 'xpos', 'ypos', 'radius', 'zRotation', 'decodedId', 'frame_idx', 'timestamp', 'cam_id', 'fc_id']
)

In [7]:
def get_dataframe2(fcs):
    """
    Converts framecontainer(s) to a dataframe. Uses fixed columns.
    :param fcs: One or multiple framecontainer
    :return: Pandas Dataframe
    """
    if not isinstance(fcs, (tuple, list)):
        fcs = fcs,

    tpls = []
    for fc in fcs:
        for f in fc.frames:
            for d in f.detectionsUnion.detectionsDP:
                d = Detection(d.idx, d.xpos, d.ypos, d.radius, d.zRotation, list(d.decodedId), f.frameIdx, f.timestamp, fc.camId, fc.id)
                tpls.append(d)

    df = pd.DataFrame(tpls)
    return df

In [8]:
def getIDs(m, d, h, files):
    a = np.zeros(2**12, dtype='int32')
    
    for f in files:
        fc = load_frame_container(f)
        df = get_dataframe2(fc)
        df = calcIds(df, 0.95)
        ids = list(df.id)
        for i in ids:
            a[i] += 1
    return (m,d,h,a)

In [ ]:
# /mnt/data/all

In [74]:
def run(cpu = 8):
    
    pool = multiprocessing.Pool(cpu)
    
    repo = Repository("data/1d/")
    
    file_list = []
    for f in repo.iter_fnames():
        string = f.split('2015')
        datum = string[1].split("/")
        file_list.append([datum[1], datum[2], datum[3], f])
    
    # DataFrame with all files
    df = DataFrame(file_list, columns=['m', 'd', 'h', 'file'])
    
    # Group by hours
    gr = df.groupby(by=['m','d','h'])
    
    # Pro Gruppe einen Task erstellen
    tasks = []

    for group in gr:
        files = list(group[1].file)
        tasks.append((group[0][0], group[0][1], group[0][2], files))
        
    results = [pool.apply_async( getIDs, t ) for t in tasks[:2]]
    
    # Write results straight to DB
    conn = sqlite3.connect('ids.db')
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS IDS''')
    c.execute('''CREATE TABLE IDS
           (MONTH   CHARACTER(20)   NOT NULL,
           DAY   CHARACTER(20)   NOT NULL,
           HOUR  CHARACTER(20)   NOT NULL,
           ID   INT   NOT NULL,
           COUNT   INT   NOT NULL);''')
    
    for result in results:
        
        res = result.get()
        m = res[0]
        d = res[1]
        h = res[2]
        
        for e, r in enumerate(res[3]):
            c.execute("insert into ids (month, day, hour, id, count) values (?, ?, ?, ?, ?)",
            (m, d, h, e, int(r)))

    pool.close()
    pool.join()
    
    counting = c.execute('SELECT count(*) FROM IDS')
    
    conn.commit()
    conn.close()

In [75]:
l = run(8)

In [76]:
conn = sqlite3.connect('ids.db')

In [77]:
c = conn.cursor()

In [80]:
num = 0
for e in c.execute('SELECT count(*) FROM IDS'):
    if (num < 100):
        print(e)
    num += 1
    

(8192,)
